# Create monthly CESM atmospheric deposition files

NCAR CESM output:
Community Earth System Model (Community Atmosphere Model - CAM https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1002/2013MS000279) output: https://www.earthsystemgrid.org/; CESM1 CAM5 BGC Large Ensemble Atmosphere Post Processed Data, Monthly Averages.

Specific run output: https://www.earthsystemgrid.org/dataset/ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.atm.proc.monthly_ave.html

In [8]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import matplotlib
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [9]:
print('Run id: ucar.cgd.ccsm4.CESM_CAM5_BGC_LE')

Run id: ucar.cgd.ccsm4.CESM_CAM5_BGC_LE


For Mode MAM3 (https://www.geosci-model-dev.net/5/709/2012/):

- a1 --- Aitken mode --- 0.015-0.053 μm
- a2 --- Accumulation mode --- 0.058-0.27 μm
- a3 --- Coarse mode --- 0.80-3.65 μm

##### Load files

In [10]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
tmask = np.array(tmask[0,:,:,:])

mesh  = nc.Dataset('/data/brogalla/old/meshmasks/ANHA12_mesh1.nc')
mlons = np.array(mesh.variables['nav_lon'])
mlats = np.array(mesh.variables['nav_lat'])
mlons = np.array(mlons)
mlats = np.array(mlats)

cond = (tmask > 0.1) 
Z_masked = np.ma.masked_where(cond, tmask) 

##### Functions

In [11]:
def interp_np(nav_lon, nav_lat, var_in, lon_ANHA12, lat_ANHA12):
    ''' Interpolate some field to ANHA12 grid.
        The function is based on the bilinear interpolation in scipy, griddata 
        =======================================================================
            nav_lon, nav_lat        : input field lon/lat
            lon_ANHA12, lat_ANHA12  : ANHA12 defined lons/lats
            var_in                  : 2-D model variable
    '''
    from scipy.interpolate import griddata
    LatLonPair = (nav_lon.flatten(), nav_lat.flatten())
    a = nav_lon.flatten()
    b = nav_lat.flatten()
    c = var_in.flatten()
#     print(a.shape, b.shape, c.shape, lon_ANHA12.shape, lat_ANHA12.shape)
    var_out = griddata(LatLonPair, var_in.flatten(), (lon_ANHA12, lat_ANHA12), method='linear')
    # Take nearest neighbour interpolation to fill nans
    var_fill = griddata(LatLonPair, var_in.flatten(), (lon_ANHA12, lat_ANHA12), method='nearest')
    
    var_out[np.isnan(var_out)] = var_fill[np.isnan(var_out)]
    return var_out

In [12]:
def save_file(filename, field1, field2, field3):
    ncd = nc.Dataset(filename, 'w', zlib=True)
#     nc_tools.init_dataset_attrs(
#         ncd,
#         title='dust input field',
#         notebook_name='',
#         nc_filepath='./monthly_CESM_forcing.nc',
#         comment='CESM ucar.cgd.ccsm4.CESM_CAM5_BGC_LE dust deposition field')

    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # variables
    fine_dust             = ncd.createVariable('fdust', 'float64', ('y','x'))
    fine_dust.units       = 'kg/m2/s'
    fine_dust.long_name   = 'Fine dust deposition flux'  
    fine_dust.coordinates = 'nav_lon nav_lat'
    fine_dust[:]          = field1
    
    coarse_dust             = ncd.createVariable('cdust', 'float64', ('y','x'))
    coarse_dust.units       = 'kg/m2/s'
    coarse_dust.long_name   = 'Coarse dust deposition flux'  
    coarse_dust.coordinates = 'nav_lon nav_lat'
    coarse_dust[:]          = field2
    
    black_c             = ncd.createVariable('bc', 'float64', ('y','x'))
    black_c.units       = 'kg/m2/s'
    black_c.long_name   = 'Black carbon deposition flux'  
    black_c.coordinates = 'nav_lon nav_lat'
    black_c[:]          = field3
    
    print('saved ', filename)

    ncd.close()
    return

In [18]:
def load_data():
    folder = '/ocean/brogalla/GEOTRACES/data/NCAR/'

    dst_a1D = nc.Dataset(folder+'merged_dst_a1DDF.nc')
    dst_a1S = nc.Dataset(folder+'merged_dst_a1SFWET.nc')
    dst_a3D = nc.Dataset(folder+'merged_dst_a3DDF.nc')
    dst_a3S = nc.Dataset(folder+'merged_dst_a3SFWET.nc')
    bc_a1D  = nc.Dataset(folder+'merged_bc_a1DDF.nc')
    bc_a1S  = nc.Dataset(folder+'merged_bc_a1SFWET.nc')
    
    dst_a1DDF   = np.array(dst_a1D.variables['dst_a1DDF'])
    dst_a1SFWET = np.array(dst_a1S.variables['dst_a1SFWET'])
    dst_a3DDF   = np.array(dst_a3D.variables['dst_a3DDF'])
    dst_a3SFWET = np.array(dst_a3S.variables['dst_a3SFWET'])
    BC_a1DDF    = np.array(bc_a1D.variables['bc_a1DDF'])
    BC_a1SFWET  = np.array(bc_a1S.variables['bc_a1SFWET'])
    
    # zero negative deposition fluxes:
    dst_a1DDF[dst_a1DDF < 0] = 0
    dst_a1SFWET[dst_a1SFWET < 0] = 0
    dst_a3DDF[dst_a3DDF < 0] = 0
    dst_a3SFWET[dst_a3SFWET < 0] = 0
    BC_a1DDF[BC_a1DDF < 0] = 0
    BC_a1SFWET[BC_a1SFWET < 0] = 0
    
    lon  = dst_a1D.variables['lon']
    lat  = dst_a1D.variables['lat']
    date = dst_a1D.variables['date']
    
    lon = np.array(lon)
    for i in range(0,len(lon)):
        if lon[i] >= 180:
            lon[i] = -360+lon[i]
            
    # change lons and lats array dimensions: (192x288)
    lons, lats = np.meshgrid(lon,lat)
    lons=np.array(lons)
    lats=np.array(lats)    
    
    dst_a1 = np.add(dst_a1DDF[:,:,:], dst_a1SFWET[:,:,:])
    dst_a3 = np.add(dst_a3DDF[:,:,:], dst_a3SFWET[:,:,:])
    bc_a1 = np.add(BC_a1DDF[:,:,:], BC_a1SFWET[:,:,:])
    
    return date, lons, lats, dst_a1, dst_a3, bc_a1

In [19]:
def find_dates(file_year):
    file_date_start = file_year*10000 + 101
    file_date_end = file_year*10000 + 1201
    
    start_index = []
    end_index = []

    for i in range(0,len(date)):
        if date[i] == file_date_start:
            start_index = i
        elif date[i] == file_date_end:
            end_index = i

    # print('start index: ', start_index)
    # print('end index: ', end_index)
    
    return start_index, end_index

In [23]:
def data_to_ANHA12(file_year, savefiles=False):
    
    si, ei = find_dates(file_year)
    dusta1 = dust_a1[si:ei+1,:,:]
    dusta3 = dust_a3[si:ei+1,:,:]
    bca1   = bc_a1[si:ei+1,:,:]
    print('Check that output is of the correct dimensions: ', dusta1.shape, dusta3.shape, bca1.shape)
    
    interp_dst_a1 = np.empty((12, 2400, 1632))
    interp_dst_a3 = np.empty((12, 2400, 1632))
    interp_bc_a1  = np.empty((12, 2400, 1632))

    # loop over the months:
    for i in range(0,12):
        interp_dst_a1[i,:,:] = interp_np(lons, lats, dusta1[i,:,:], mlons, mlats)
        interp_dst_a3[i,:,:] = interp_np(lons, lats, dusta3[i,:,:], mlons, mlats)
        interp_bc_a1[i,:,:]  = interp_np(lons, lats, bca1[i,:,:], mlons, mlats)
        
    if savefiles:
        location = '/ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/'
        
        for i in range(1,13):
            if i < 10:
                save_file(location+'atm_flux_y'+str(file_year)+'m0'+str(i)+'.nc',interp_dst_a1[i-1,:,:],\
                          interp_dst_a3[i-1,:,:], interp_bc_a1[i-1,:,:])
            else:
                save_file(location+'atm_flux_y'+str(file_year)+'m'+str(i)+'.nc',interp_dst_a1[i-1,:,:],\
                          interp_dst_a3[i-1,:,:], interp_bc_a1[i-1,:,:])    
    
    return interp_dst_a1, interp_dst_a3, interp_bc_a1

Interpolate to ANHA12 grid:

In [24]:
date, lons, lats, dust_a1, dust_a3, bc_a1 = load_data()

In [34]:
for year in np.arange(2007,2020,1):
    print(year)
    interp_dst_a1, interp_dst_a3, interp_bc_a1 = data_to_ANHA12(year, savefiles=True)

2007
Check that output is of the correct dimensions:  (12, 192, 288) (12, 192, 288) (12, 192, 288)
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m01.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m02.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m03.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m04.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m05.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m06.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m07.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m08.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2007m09.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/at

saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2013m12.nc
2014
Check that output is of the correct dimensions:  (12, 192, 288) (12, 192, 288) (12, 192, 288)
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m01.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m02.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m03.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m04.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m05.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m06.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m07.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/atm_flux_y2014m08.nc
saved  /ocean/brogalla/GEOTRACES/data/paper1-forcing-files/atmospheric/at

In [29]:
imin, imax = 1480, 2180
jmin, jmax = 160, 800
isize, jsize = imax-imin, jmax-jmin

In [32]:
print('NCAR dust flux from the atmosphere forcing field -----')
print('Maximum value: ', np.amax(interp_dst_a1[0,imin:imax,jmin:jmax]+interp_dst_a3[0,imin:imax,jmin:jmax]))
print('Minimum value: ', np.amin(interp_dst_a1[0,imin:imax,jmin:jmax]+interp_dst_a3[0,imin:imax,jmin:jmax]))
print('Average value: ', np.average(interp_dst_a1[0,imin:imax,jmin:jmax]+interp_dst_a3[0,imin:imax,jmin:jmax]))

NCAR dust flux from the atmosphere forcing field -----
Maximum value:  4.793804970368727e-14
Minimum value:  3.357708424140673e-15
Average value:  1.0886650673357239e-14
